In [37]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [38]:
train = pd.read_csv('Train_UWu5bXk.csv')
test = pd.read_csv('Test_u94Q5KV.csv')
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True)
print (train.shape, test.shape, data.shape)

(8523, 13) (5681, 12) (14204, 13)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [39]:
data.head(3)

Item_Fat_Content Item_Identifier  Item_MRP  Item_Outlet_Sales    Item_Type  \
0          Low Fat           FDA15  249.8092          3735.1380        Dairy   
1          Regular           DRC01   48.2692           443.4228  Soft Drinks   
2          Low Fat           FDN15  141.6180          2097.2700         Meat   

   Item_Visibility  Item_Weight  Outlet_Establishment_Year Outlet_Identifier  \
0         0.016047         9.30                       1999            OUT049   
1         0.019278         5.92                       2009            OUT018   
2         0.016760        17.50                       1999            OUT049   

  Outlet_Location_Type Outlet_Size        Outlet_Type source  
0               Tier 1      Medium  Supermarket Type1  train  
1               Tier 3      Medium  Supermarket Type2  train  
2               Tier 1      Medium  Supermarket Type1  train

In [40]:
data.apply(lambda x: sum(x.isnull()))

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
source                          0
dtype: int64

In [41]:
categorical_columns = [x for x in data.dtypes.index if data.dtypes[x]=='object']
categorical_columns = [x for x in categorical_columns if x not in ['Item_Identifier','Outlet_Identifier','source']]
for col in categorical_columns:
    print ('\nFrequency of Categories for varible %s'%col)
    print (data[col].value_counts())


Frequency of Categories for varible Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for varible Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for varible Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Frequency of Categories for varible Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

F

In [42]:
data.Item_Weight = data.Item_Weight.fillna(data.Item_Weight.mean())

In [43]:
from scipy.stats import mode

outlet_size_mode = data.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x:mode(x).mode[0]) )

miss_bool = data['Outlet_Size'].isnull() 

data.loc[miss_bool,'Outlet_Size'] = data.loc[miss_bool,'Outlet_Type'].apply(lambda x: outlet_size_mode[x])
print (sum(data['Outlet_Size'].isnull()))

0


In [44]:
data.Item_Visibility=data.Item_Visibility.replace(0.0, data.Item_Visibility.mean())

In [45]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [46]:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

In [47]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print (data['Item_Fat_Content'].value_counts())

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64


In [48]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [49]:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

In [50]:
data.dtypes

Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
source                        object
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_0                       uint8
Outlet_1                       uint8
O

In [51]:
#Drop the columns which have been converted to different types:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

#Export files as modified versions:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

In [52]:
train = pd.read_csv('train_modified.csv')
test = pd.read_csv('test_modified.csv')

train = train.drop(['Item_Identifier','Outlet_Identifier'],axis=1)

Y_train = train.pop('Item_Outlet_Sales')
X_train = train.iloc[:,:]

test = test.drop(['Item_Identifier','Outlet_Identifier'],axis=1)
X_test = test.iloc[:,:]


In [53]:
from catboost import CatBoostRegressor

In [54]:
modelc=CatBoostRegressor(iterations=200,
                            learning_rate=0.6,
                            depth=8,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            random_seed=99,
                            od_type='Iter',
                            od_wait=50)
modelc.fit(X_train, Y_train, use_best_model=True, verbose=False)

You should provide test set for use best model. use_best_model parameter swiched to false value.


In [55]:
Y_pred = modelc.predict(X_test)

In [56]:
for item in Y_pred:
    print (item)

1719.815833451504
1590.115605638124
873.9893478130828
2417.2134337825164
5835.845351647501
1677.9211183952395
626.6076329239919
1898.9681770672346
1491.3204982816073
2969.763940991794
1715.931353623446
1528.6733345718421
527.8291637516078
2312.7618061227918
755.6039696149395
2428.4851576590495
3201.038085915005
2774.473645899697
2920.4558807525837
518.7159038292018
2806.9918504385228
3090.5303371441714
720.8437807091929
388.70111577524364
3392.5408079093672
772.6670617298471
796.9364019238745
2504.7010609186987
3724.4295807137223
1914.094903958687
90.56389918437922
3008.747784694957
557.9548294430062
5208.2111245069855
2063.6932947989794
5562.466936978274
2130.0383201204118
278.43136280565113
3075.814401682599
2659.362472246363
2420.5450078994327
132.74239496119742
2520.3899103187014
2106.4127308530014
459.10063394597273
1781.083361832584
95.26786206106374
3336.2746626011135
4010.0313316315905
1887.0869786856242
1172.5165055344874
1755.7045771416592
3479.63268509679
2547.723420401011
5

2011.2069025064911
370.6850909113932
1511.3302762671694
3198.3952385184884
3126.977779723299
1159.0519904493613
2617.0321701413386
1125.3917909727327
5247.03451373894
834.0455196332324
3288.7397747849823
550.5334407706749
5012.3599863751515
2228.431672166843
421.47803434672556
2098.154066070848
654.2380748578935
1753.1717821019643
2372.495337950968
3157.691287519294
2123.079865303634
2622.001265280782
3420.3710994491717
3035.1624539453237
1614.573266298541
933.0597592665526
2542.4967502249456
1697.5211734122345
592.8330126179154
3163.1404121646806
524.6383980218222
2322.695151091451
1086.6634919452001
4211.814645682861
3527.3793453956155
3487.6747257674806
2349.9632857548204
2983.6807063704578
1029.5944275695165
1344.9641284961626
2452.683369179907
3595.784582940742
903.2275641814382
6014.011578199619
2977.7066746640985
2245.9847148971007
1290.1893268762822
3263.8887417450587
124.89149603394912
2307.0479737251703
3750.2135118115652
585.8990076563774
3048.968722801553
180.08335730052698

221.18100235235875
4397.782797560717
295.18499573351056
1431.4926803813034
2100.304595710491
2339.1657511580916
1649.7172302537465
1842.9750596022616
3272.065086241714
852.4181056567373
3205.79925401599
3375.1392869521233
4608.5379287945125
2451.4166216719004
1340.1731065721306
2742.6992272619436
818.6052883055299
1644.252249485925
496.7349392063083
2091.2424317803493
5174.3400347810375
4081.37842775386
3883.386503109169
4630.354847035921
2653.6612163437944
1578.6877585247626
1266.0905170438525
954.0774306301032
2746.6073308954087
1279.3926386192559
2924.098829983247
461.38060735567575
1245.5838412980324
2052.0910622996003
259.5907912011877
1783.8013023164024
1747.95651974534
2448.8658328100482
1836.664002290123
246.30565639505053
2014.2653355732489
2547.879197752694
1709.141989668395
69.14582662791034
836.9719082589276
1644.9875953777735
4669.009969764812
1729.658308374014
1507.6799843536724
2332.4444171586774
1555.867413327518
3513.288297093186
2906.7919016647306
1817.5387301665733
2

2955.522730553228
1836.6021291437028
5157.829437100958
350.098301247052
4165.956399585894
2223.733561708024
2460.490667529717
4501.364115651559
2059.153302099503
4290.410036915506
1451.0865349863984
209.46581528235976
3146.0260907905895
4346.9116810234755
2013.6570715037806
1342.8787424777117
2374.1652821820203
147.74747865441336
858.0866260358403
740.7222002243128
5911.765718325423
3010.893309486896
3280.489021330627
3190.21299042575
3455.2438484687527
2923.7831676039427
2212.263284771758
554.333390598571
198.26788999709217
1852.8120728366544
1493.765800405095
188.82610531004562
3222.6532366340552
1775.7682648338807
2956.681570061344
1579.0373366301164
1765.272547474956
2886.1687399033176
85.71042625728975
3217.551357937233
3853.1162996889825
2716.6923409980013
650.8400454750098
3004.8777238935872
1744.6451250966556
1390.7390129468424
2044.1651602392856
1914.1268579758812
2657.4484721425647
2012.2260766611735
607.3295641094622
1452.3569327815776
1971.2750349891926
1237.7271193767526
1

2483.797596187779
3516.40294296292
2765.154791720196
1443.3082881054634
1305.6700037662317
1847.2481289977025
859.4208470033096
644.6417665446796
2258.60088873904
3502.864998231806
697.6098549828549
2612.280893922962
2302.089512512389
658.1401226473592
1824.277412636382
212.75866977769223
2519.4939914028346
1944.260826118256
250.59746562777704
475.33103656275034
1309.6512116001697
2925.5333376404983
952.2420173352125
3330.112330677401
3053.3110733176936
3050.908844784861
2575.499165349595
1801.862205888206
2896.5582694179966
2902.5130405747004
4316.6232059405875
563.7848361419661
1620.9827484985565
2391.7326181260296
881.1810970547183
236.975324043712
3490.077470938523
2284.863217543512
3025.694184648718
1861.226319638386
2357.062797626169
2117.0832504697146
2601.394890095496
3707.9634553153724
6208.586288185258
3779.5850757959247
3049.070933811231
308.34909282560375
3703.7196726376533
2827.444880677829
7918.344046628317
2505.784005800589
1505.256640199999
1039.1327608534893
2422.16195

In [57]:
sample_file = pd.read_csv('SampleSubmission_TmnO39y.csv')
sample_file

Item_Identifier Outlet_Identifier  Item_Outlet_Sales
0              FDW58            OUT049        1686.592455
1              FDW14            OUT017        1377.134667
2              NCN55            OUT010         660.745973
3              FDQ58            OUT017        2148.875553
4              FDY38            OUT027        6186.595442
5              FDH56            OUT046        1901.137425
6              FDL48            OUT018         677.033862
7              FDC48            OUT027        1787.636684
8              FDN33            OUT045         967.074500
9              FDA36            OUT017        2474.439647
10             FDT44            OUT017        1595.317327
11             FDQ56            OUT045        1587.146145
12             NCC54            OUT019         740.430127
13             FDU11            OUT049        1880.049724
14             DRL59            OUT013         900.724504
15             FDM24            OUT049        2723.418584
16             FDI57            OUT045        3824.427833
17             DRC12            OUT018        3323.945973
18             NCM42            OUT027        3422.805167
19             FDA46            OUT010         452.120569
20             FDA31            OUT013        2854.520656
21             NCJ31            OUT035        3546.982920
22             FDG52            OUT046         721.569829
23             NCL19            OUT019         381.781825
24             FDS10            OUT035        2891.454398
25             FDX22            OUT010         420.283224
26             NCF19            OUT035         703.369278
27             NCE06            OUT046        2100.205573
28             DRC27            OUT046        4263.183980
29             FDE21            OUT035        1886.774304
...              ...               ...                ...
5651           FDD23            OUT013        3442.252580
5652           FDP32            OUT045        1804.886956
5653           FDO31            OUT035        1288.522740
5654           FDQ57            OUT013        2765.503196
5655           FDX32            OUT027        4038.990962
5656           FDC39            OUT027        5589.300209
5657           FDU55            OUT046        5110.971331
5658           FDL45            OUT013        2379.756835
5659           DRK37            OUT049        4108.585220
5660           FDK22            OUT013        3121.936200
5661           DRG37            OUT027        4981.570075
5662           FDK22            OUT017        4180.128456
5663           DRH36            OUT018         961.372831
5664           DRC36            OUT018        3167.095598
5665           DRE03            OUT027         962.565218
5666           FDF34            OUT046        2731.057125
5667           FDZ22            OUT046        1390.426456
5668           FDC44            OUT010         310.765176
5669           FDN31            OUT027        5070.750958
5670           FDO03            OUT017        4726.677624
5671           FDA01            OUT049         819.121635
5672           NCH42            OUT049        4340.701258
5673           FDF46            OUT018        1589.730660
5674           DRL35            OUT046         799.008422
5675           FDW46            OUT049        1171.596155
5676           FDB58            OUT046        2314.314747
5677           FDD47            OUT018        2784.744816
5678           NCO17            OUT045        1770.180618
5679           FDJ26            OUT017        4396.017133
5680           FDU37            OUT045        1449.694762

[5681 rows x 3 columns]

In [58]:
sample_file['Item_Outlet_Sales'] = Y_pred
print(sample_file)
sample_file.to_csv('submission11.csv', index=False)

     Item_Identifier Outlet_Identifier  Item_Outlet_Sales
0              FDW58            OUT049        1719.815833
1              FDW14            OUT017        1590.115606
2              NCN55            OUT010         873.989348
3              FDQ58            OUT017        2417.213434
4              FDY38            OUT027        5835.845352
5              FDH56            OUT046        1677.921118
6              FDL48            OUT018         626.607633
7              FDC48            OUT027        1898.968177
8              FDN33            OUT045        1491.320498
9              FDA36            OUT017        2969.763941
10             FDT44            OUT017        1715.931354
11             FDQ56            OUT045        1528.673335
12             NCC54            OUT019         527.829164
13             FDU11            OUT049        2312.761806
14             DRL59            OUT013         755.603970
15             FDM24            OUT049        2428.485158
16            

In [59]:
'''
Changes:
1) Append a column called source to both train and test data
2) sum(x.isnull()
3) Extract categorical columns - data.dtypes[x]=='object' - except Item identifier, outlet identifier, source
4) Replace empty values in Outlet_Type with mode of Outlet_Type - mode(x).mode[0]
5) Item_Visibility - find mean. Should not be zero
6) Extract first two letters of Item_Identifier. Map Item_Type_Combined to Food, Non-Consumable, Drinks
7) Outlet_Years = 2013 - Outlet_Establishment_Year
8) Item_Fat_Content - replace LF and low fat as Low Fat, reg as Regular
9) Label Encode, OneHot Encode
10) data.drop['Item_Type','Outlet_Establishment_Year']
11) Separate test,train using source
12) test.drop['Item_Outlet_Sales','source'], train.drop['source']
13) to_csv
14) Find X_train,Y_train,X_test
15) Apply algo and determine Y_pred
'''

"\nChanges:\n1) Append a column called source to both train and test data\n2) sum(x.isnull()\n3) Extract categorical columns - data.dtypes[x]=='object' - except Item identifier, outlet identifier, source\n4) Replace empty values in Outlet_Type with mode of Outlet_Type - mode(x).mode[0]\n5) Item_Visibility - find mean. Should not be zero\n6) Extract first two letters of Item_Identifier. Map Item_Type_Combined to Food, Non-Consumable, Drinks\n7) Outlet_Years = 2013 - Outlet_Establishment_Year\n8) Item_Fat_Content - replace LF and low fat as Low Fat, reg as Regular\n9) Label Encode, OneHot Encode\n10) data.drop['Item_Type','Outlet_Establishment_Year']\n11) Separate test,train using source\n12) test.drop['Item_Outlet_Sales','source'], train.drop['source']\n13) to_csv\n14) Find X_train,Y_train,X_test\n15) Apply algo and determine Y_pred\n"